In [ ]:
#We load in the specific data needed
import polars as pl
import numpy as np
import os
from pathlib import Path
import pandas as pd
from datetime import date
import datetime
import hvplot.polars
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.interpolate import bisplrep, bisplev
from datetime import timedelta
from patsy import dmatrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV


In [ ]:
data_train_p = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_train_tech_scaled_p.parquet')
data_train_c = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_train_tech_scaled_c.parquet')

data_val_p = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_val_tech_scaled_p.parquet')
data_val_c = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_val_tech_scaled_c.parquet')

data_test_p = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_test_tech_scaled_p.parquet')
data_test_c = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_test_tech_scaled_c.parquet')

tot_data_train_c = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_train_val_tech_scaled_c.parquet')
tot_data_train_p = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_train_val_tech_scaled_p.parquet')

firm_data = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/Firm_variables/daily_firm_data_median_new.parquet')

data_train = pd.concat([data_train_p, data_train_c], axis=0)
data_val = pd.concat([data_val_p, data_val_c], axis=0)
data_test = pd.concat([data_test_p, data_test_c], axis=0)
data_train_tot = pd.concat([tot_data_train_c, tot_data_train_p], axis=0)

# # List of columns to drop
# columns_to_drop = ['trading_days_till_exp'] + list(firm_data.columns[2:])
# # columns_to_drop = ['trading_days_till_exp']

# # Drop columns from datasets if they exist
# data_train = data_train.drop(columns=columns_to_drop, errors='ignore')
# data_val = data_val.drop(columns=columns_to_drop, errors='ignore')
# data_test = data_test.drop(columns=columns_to_drop, errors='ignore')
# data_train_tot = data_train_tot.drop(columns=columns_to_drop, errors='ignore')

# Ensure that data_val and data_test have the same column order as data_train
data_val = data_val[data_train.columns]
data_test = data_test[data_train.columns]
tot_data_train = data_train_tot[data_train.columns]

data_test



In [ ]:
# Top features for Call and Put options
top_features_c = ['cp_flag', 'Ticker', 'date', 'moneyness', 'impl_volatility', 
                  'prev_day_iv', 'idiovol', 'T', 
                  'prev2_day_iv', 'moneyness', '1Y_bond', 'vol_stock']

top_features_p = ['cp_flag', 'Ticker', 'date', 'moneyness', 'impl_volatility', 
                  'prev_day_iv', 'T', 'prev2_day_iv', 
                  'idiovol', 'moneyness', 'vol_stock']

# Prepare train data for Call and Put options
data_train_c = data_train[data_train['cp_flag'] == 'C'][top_features_c]
data_train_p = data_train[data_train['cp_flag'] == 'P'][top_features_p]

# Prepare validation data for Call and Put options
data_validate_c = data_val[data_val['cp_flag'] == 'C'][top_features_c]
data_validate_p = data_val[data_val['cp_flag'] == 'P'][top_features_p]

# Prepare the total train data for Call and Put options
data_train_tot_c = tot_data_train[tot_data_train['cp_flag'] == 'C'][top_features_c]
data_train_tot_p = tot_data_train[tot_data_train['cp_flag'] == 'P'][top_features_p]

# Prepare test data for Call and Put options
data_test_c = data_test[data_test['cp_flag'] == 'C'][top_features_c]
data_test_p = data_test[data_test['cp_flag'] == 'P'][top_features_p]

In [ ]:
# Prepare train, validation, and test data for Call options
train_x_c = data_train_c.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])
train_y_c = data_train_c['impl_volatility']

validate_x_c = data_validate_c.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])
validate_y_c = data_validate_c['impl_volatility']

combined_x_c = data_train_tot_c.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])
combined_y_c = data_train_tot_c['impl_volatility']

test_x_c = data_test_c.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])
test_y_c = data_test_c['impl_volatility']

# Prepare train, validation, and test data for Put options
train_x_p = data_train_p.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])
train_y_p = data_train_p['impl_volatility']

validate_x_p = data_validate_p.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])
validate_y_p = data_validate_p['impl_volatility']

combined_x_p = data_train_tot_p.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])
combined_y_p = data_train_tot_p['impl_volatility']

test_x_p = data_test_p.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])
test_y_p = data_test_p['impl_volatility']

In [ ]:
# import numpy as np
# import keras.backend as K
# from keras import regularizers
# from keras.layers import Dense, Dropout
# from keras.models import Sequential
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import mean_squared_error, r2_score
# from scikeras.wrappers import KerasRegressor  # Use SciKeras instead
# from keras.optimizers import RMSprop

# # Define the model function with variable neurons, layers, and dropout rate
# def create_model(neurons=32, layers=1, dropout_rate=0.0, bias=0.01, activity=0.01):
#     model = Sequential()
#     # Input layer (first hidden layer)
#     model.add(Dense(neurons, activation='relu', input_dim=train_x.shape[1],
#                     bias_regularizer=regularizers.L2(bias),
#                     activity_regularizer=regularizers.L2(activity)))
#     model.add(Dropout(dropout_rate))  # Dropout layer after the first hidden layer

#     # Additional hidden layers
#     for _ in range(layers - 1):
#         model.add(Dense(neurons, activation='relu',
#                         bias_regularizer=regularizers.L2(bias),
#                         activity_regularizer=regularizers.L2(activity)))
#         model.add(Dropout(dropout_rate))

#     # Output layer
#     model.add(Dense(1, activation='linear'))  # Output layer for regression (linear)

#     # Compile the model
#     model.compile(optimizer=RMSprop(learning_rate=0.01),
#                   loss='mean_squared_error',
#                   metrics=['mse'])
#     return model

# # Wrapping the model in KerasRegressor
# model = KerasRegressor(build_fn=create_model, verbose=0)

# # Define the parameter grid for trials
# param_grid = {
#     'neurons': [8, 16, 32, 64],   # Number of neurons in each hidden layer
#     'layers': [1, 2, 3, 4],       # Number of hidden layers
#     'dropout_rate': [0, 0.2, 0.5], # Dropout rate
#     'batch_size': [32, 64],       # Batch size for training
#     'epochs': [50, 100],          # Number of epochs
# }

# # Function to perform hyperparameter tuning, retrain the model, and test
# def train_and_evaluate(train_x, train_y, validate_x, validate_y, combined_x, combined_y, test_x, test_y):
#     # Initialize GridSearchCV with the model, parameter grid, and scoring
#     grid_search = GridSearchCV(estimator=model,
#                                param_grid=param_grid,
#                                scoring='neg_mean_squared_error',  # Scoring based on MSE
#                                cv=3,  # 3-fold cross-validation
#                                verbose=1)  # Verbose for tracking progress

#     # Hyperparameter tuning on training and validation sets
#     print("Running hyperparameter tuning...")
#     grid_search.fit(train_x, train_y, validation_data=(validate_x, validate_y))

#     # Get the best estimator and parameters
#     best_model = grid_search.best_estimator_
#     best_params = grid_search.best_params_

#     print(f"Best Parameters: {best_params}")

#     # Retrain on the combined training and validation set
#     best_model.fit(combined_x, combined_y)

#     # Make predictions on the test set
#     predictions = best_model.predict(test_x)

#     # Calculate R² and RMSE for the best model
#     r2 = r2_score(test_y, predictions)
#     rmse = np.sqrt(mean_squared_error(test_y, predictions))

#     # Print the results
#     print(f"R²: {r2:.4f}")
#     print(f"RMSE: {rmse:.4f}")
#     return best_model

# # Call the function for Call options data
# print("\nEvaluating Call options...")
# best_model_call = train_and_evaluate(train_x_c, train_y_c, validate_x_c, validate_y_c, combined_x_c, combined_y_c, test_x_c, test_y_c)

# # Call the function for Put options data
# print("\nEvaluating Put options...")
# best_model_put = train_and_evaluate(train_x_p, train_y_p, validate_x_p, validate_y_p, combined_x_p, combined_y_p, test_x_p, test_y_p)



In [ ]:
import numpy as np
from keras import regularizers
from keras.layers import Dense, Dropout, Input
from keras.models import Sequential
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from scikeras.wrappers import KerasRegressor
from keras.optimizers import RMSprop

# Define the model function with variable neurons, layers, and dropout rate
def create_model(input_dim, neurons=32, layers=1, dropout_rate=0.0, bias=0.01, activity=0.01):
    model = Sequential()
    # Input layer using Input instead of input_dim argument
    model.add(Input(shape=(input_dim,)))  # Define the input shape explicitly

    # First hidden layer
    model.add(Dense(neurons, activation='relu',
                    bias_regularizer=regularizers.L2(bias),
                    activity_regularizer=regularizers.L2(activity)))
    model.add(Dropout(dropout_rate))  # Dropout layer after the first hidden layer

    # Additional hidden layers
    for _ in range(layers - 1):
        model.add(Dense(neurons, activation='relu',
                        bias_regularizer=regularizers.L2(bias),
                        activity_regularizer=regularizers.L2(activity)))
        model.add(Dropout(dropout_rate))

    # Output layer
    model.add(Dense(1, activation='linear'))  # Output layer for regression (linear)

    # Compile the model
    model.compile(optimizer=RMSprop(learning_rate=0.01),
                  loss='mean_squared_error',
                  metrics=['mse'])
    return model

# Wrapping the model in KerasRegressor
def create_keras_regressor(input_dim):
    return KerasRegressor(model=create_model, input_dim=input_dim, verbose=1)  # Set verbose=1 for model fit

# Define the parameter grid for trials
param_grid = {
    'model__neurons': [8, 16, 32, 64],    # Number of neurons in each hidden layer
    'model__layers': [1, 2, 3, 4],        # Number of hidden layers
    'model__dropout_rate': [0, 0.2, 0.5], # Dropout rate
    'batch_size': [32],               # Batch size for training
    'epochs': [50],                  # Number of epochs
}

# Function to perform hyperparameter tuning, retrain the model, and test
def train_and_evaluate(train_x, train_y, validate_x, validate_y, combined_x, combined_y, test_x, test_y):
    # Get input dimension from the training data
    input_dim = train_x.shape[1]
    
    # Create KerasRegressor with the correct input dimension
    model = create_keras_regressor(input_dim)

    # Initialize GridSearchCV with the model, parameter grid, and scoring
    grid_search = GridSearchCV(estimator=model,
                               param_grid=param_grid,
                               scoring='neg_mean_squared_error',  # Scoring based on MSE
                               cv=3,  # 3-fold cross-validation
                               verbose=3)  # Verbose=3 for detailed progress tracking

    # Hyperparameter tuning using validation data
    print("Running hyperparameter tuning with validation data...")
    grid_search.fit(train_x, train_y,
                    validation_data=(validate_x, validate_y),  # Use validation set for tuning
                    verbose=1)

    # Get the best estimator and parameters
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_

    print(f"Best Parameters: {best_params}")

    # Retrain on the combined training and validation set with verbose output
    best_model.fit(combined_x, combined_y, verbose=1)  # Verbose=1 for training progress tracking

    # Make predictions on the test set
    predictions = best_model.predict(test_x)

    # Calculate R² and RMSE for the best model
    r2 = r2_score(test_y, predictions)
    rmse = np.sqrt(mean_squared_error(test_y, predictions))

    # Print the results
    print(f"R²: {r2:.4f}")
    print(f"RMSE: {rmse:.4f}")
    return best_model

# Call the function for Call options data
print("\nEvaluating Call options...")
best_model_call = train_and_evaluate(train_x_c, train_y_c, validate_x_c, validate_y_c, combined_x_c, combined_y_c, test_x_c, test_y_c)

# Call the function for Put options data
print("\nEvaluating Put options...")
# best_model_put = train_and_evaluate(train_x_p, train_y_p, validate_x_p, validate_y_p, combined_x_p, combined_y_p, test_x_p, test_y_p)


In [ ]:
# import keras.backend as K
# from keras import regularizers
# from tensorflow import keras
# from keras.layers import Dense
# from keras.models import Sequential, load_model
# from sklearn.model_selection import GridSearchCV
# from keras.wrappers.scikit_learn import KerasRegressor
# from keras.optimizers import RMSprop
# from keras.optimizers import Adam

# # define the model
# def create_model(bias, activity):
#   model = Sequential()
#   model.add(Dense(units=32,
#                   activation='relu',
#                   input_dim= train_x.shape[1],
#                   # # kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-5),
#                   bias_regularizer= regularizers.L2(bias),
#                   activity_regularizer= regularizers.L2(activity)
#                   ))
#   model.add(Dense(units=32, activation= 'relu',
#                   bias_regularizer=regularizers.L2(bias),
#                   activity_regularizer=regularizers.L2(activity) #activation
#                   ))
#   model.add(Dense(units=1, activation= 'linear')) #miss linear

#   # compile the model
#   model.compile(optimizer= RMSprop(learning_rate=0.01), #'rmsprop',#Optimizer_trial,
#                 loss= 'mean_squared_error',
#                 metrics=['mse'])
#   return model

# # Wrapping the model in KerasRegressor
# model = KerasRegressor(build_fn=create_model, verbose=0)

# # Define the parameter grid for trials
# param_grid = {
#     'neurons': [8, 16, 32, 64],   # Number of neurons in each hidden layer
#     'layers': [1, 2, 3, 4],       # Number of hidden layers
#     'dropout_rate': [0, 0.2, 0.5], # Dropout rate
#     'batch_size': [32, 64],       # Batch size for training
#     'epochs': [50, 100],          # Number of epochs
# }



In [ ]:
# import numpy as np
# import keras.backend as K
# from keras import regularizers
# from keras.layers import Dense, Dropout
# from keras.models import Sequential
# from sklearn.model_selection import GridSearchCV, train_test_split
# from sklearn.metrics import mean_squared_error, r2_score
# from keras.wrappers.scikit_learn import KerasRegressor
# from keras.optimizers import RMSprop

# # Define the model function with variable neurons, layers, and dropout rate
# def create_model(neurons=32, layers=1, dropout_rate=0.0, bias=0.01, activity=0.01):
#     model = Sequential()
#     # Input layer (first hidden layer)
#     model.add(Dense(neurons, activation='relu', input_dim=train_x.shape[1],
#                     bias_regularizer=regularizers.L2(bias),
#                     activity_regularizer=regularizers.L2(activity)))
#     model.add(Dropout(dropout_rate))  # Dropout layer after the first hidden layer

#     # Additional hidden layers
#     for _ in range(layers - 1):
#         model.add(Dense(neurons, activation='relu',
#                         bias_regularizer=regularizers.L2(bias),
#                         activity_regularizer=regularizers.L2(activity)))
#         model.add(Dropout(dropout_rate))

#     # Output layer
#     model.add(Dense(1, activation='linear'))  # Output layer for regression (linear)

#     # Compile the model
#     model.compile(optimizer=RMSprop(learning_rate=0.01),
#                   loss='mean_squared_error',
#                   metrics=['mse'])
#     return model

# # Wrapping the model in KerasRegressor
# model = KerasRegressor(build_fn=create_model, verbose=0)

# # Define the parameter grid for trials
# param_grid = {
#     'neurons': [8, 16, 32, 64],   # Number of neurons in each hidden layer
#     'layers': [1, 2, 3, 4],       # Number of hidden layers
#     'dropout_rate': [0, 0.2, 0.5], # Dropout rate
#     'batch_size': [32, 64],       # Batch size for training
#     'epochs': [50, 100],          # Number of epochs
# }

# # Initialize GridSearchCV with the model, parameter grid, and scoring
# grid_search = GridSearchCV(estimator=model,
#                            param_grid=param_grid,
#                            scoring='neg_mean_squared_error',  # Scoring based on MSE
#                            cv=3,  # 3-fold cross-validation
#                            verbose=1)  # Verbose for tracking progress

# # Run the grid search
# grid_search.fit(train_x, train_y)

# # Get the best estimator and parameters
# best_model = grid_search.best_estimator_
# best_params = grid_search.best_params_

# # Make predictions using the best model
# predictions = best_model.predict(test_x)

# # Calculate R² and RMSE for the best model
# r2 = r2_score(test_y, predictions)
# rmse = np.sqrt(mean_squared_error(test_y, predictions))

# # Print the results
# print(f"Best Parameters: {best_params}")
# print(f"R²: {r2:.4f}")
# print(f"RMSE: {rmse:.4f}")


In [ ]:
# import tensorflow as tf
# print(tf.__version__)
# import time
# import itertools
# import numpy as np
# import pandas as pd
# from sklearn.metrics import mean_squared_error, r2_score
# from keras_tuner import KerasRegressor
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.optimizers import Adam
# # from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# # from keras_tuner import KerasRegressor
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# from tensorflow.keras.callbacks import EarlyStopping

# def create_model(hidden_layers=1, units=64, activation='relu', learning_rate=0.001):
#     """
#     Function to create a Keras Sequential model with the given hyperparameters.
#     """
#     model = Sequential()
#     model.add(Dense(units=units, activation=activation, input_dim=X_train.shape[1]))
    
#     for _ in range(hidden_layers - 1):
#         model.add(Dense(units=units, activation=activation))
        
#     model.add(Dense(1))  # Output layer

#     optimizer = Adam(learning_rate=learning_rate)
#     model.compile(optimizer=optimizer, loss='mean_squared_error')
    
#     return model

# def prepare_data_with_gridsearch_nn(train_data, validate_data, test_data, option_type, verbose=True):
#     """
#     Prepare the data, perform hyperparameter tuning using Year 1 (train) and Year 2 (validation),
#     retrain the model on Year 1 + Year 2, and evaluate on Year 3 (test) for Neural Networks.
    
#     Parameters:
#     train_data (pd.DataFrame): The training dataset (Year 1).
#     validate_data (pd.DataFrame): The validation dataset (Year 2).
#     test_data (pd.DataFrame): The testing dataset (Year 3).
#     option_type (str): Call or Put option type for labeling the print output.
#     verbose (bool): If True, prints progress information for hyperparameter tuning.
#     """
#     # Prepare the train, validation, and test data
#     X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features (Year 1)
#     y_train = train_data['impl_volatility']  # Training target (Year 1)

#     X_validate = validate_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Validation features (Year 2)
#     y_validate = validate_data['impl_volatility']  # Validation target (Year 2)
    
#     X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features (Year 3)
#     y_test = test_data['impl_volatility']  # Test target (Year 3)

#     # Define the hyperparameter grid for NN
#     param_grid = {
#         'hidden_layers': [1, 2, 3],                # Number of hidden layers
#         'units': [32, 64, 128],                    # Number of units per layer
#         'activation': ['relu', 'tanh'],            # Activation function
#         'learning_rate': [0.001, 0.01],            # Learning rate for Adam optimizer
#         'batch_size': [32, 64],                    # Batch size
#         'epochs': [50, 100],                       # Number of epochs
#     }

#     # Generate all combinations of hyperparameters
#     param_combinations = list(itertools.product(
#         param_grid['hidden_layers'], 
#         param_grid['units'], 
#         param_grid['activation'], 
#         param_grid['learning_rate'], 
#         param_grid['batch_size'], 
#         param_grid['epochs']
#     ))

#     total_combinations = len(param_combinations)
    
#     # Initialize variables to store the best model and best score
#     best_rmse_val = np.inf
#     best_params = None
#     best_nn_model = None

#     print(f"Running manual hyperparameter tuning for {option_type} Options with Neural Networks...")
    
#     # Iterate over all hyperparameter combinations with progress tracking
#     for i, (hidden_layers, units, activation, learning_rate, batch_size, epochs) in enumerate(param_combinations):
#         start_time = time.time()
        
#         # Create the Keras model with the current set of hyperparameters
#         model = create_model(hidden_layers=hidden_layers, units=units, activation=activation, learning_rate=learning_rate)

#         # Early stopping to prevent overfitting
#         early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

#         # Train the model on the training data (Year 1)
#         history = model.fit(X_train, y_train, 
#                             validation_data=(X_validate, y_validate),
#                             batch_size=batch_size, 
#                             epochs=epochs, 
#                             verbose=0, 
#                             callbacks=[early_stopping])

#         # Validate the model on the validation data (Year 2)
#         y_val_pred = model.predict(X_validate)
#         rmse_val = np.sqrt(mean_squared_error(y_validate, y_val_pred))

#         # Track the best performing hyperparameters based on validation RMSE
#         if rmse_val < best_rmse_val:
#             best_rmse_val = rmse_val
#             best_params = {
#                 'hidden_layers': hidden_layers,
#                 'units': units,
#                 'activation': activation,
#                 'learning_rate': learning_rate,
#                 'batch_size': batch_size,
#                 'epochs': epochs
#             }
#             best_nn_model = model

#         # Verbose output to track progress
#         if verbose:
#             elapsed_time = time.time() - start_time
#             print(f"Combination {i + 1}/{total_combinations} completed in {elapsed_time:.2f} seconds.")
#             print(f"Current RMSE (Validation): {rmse_val:.4f}")
#             print(f"Best RMSE so far: {best_rmse_val:.4f}")
    
#     print(f"\nBest Parameters for {option_type} Options with Neural Networks: {best_params}")
    
#     # Retrain the model on combined Year 1 (train) and Year 2 (validation)
#     print("Retraining the model on Year 1 and Year 2 combined...")
#     X_combined = pd.concat([X_train, X_validate])
#     y_combined = pd.concat([y_train, y_validate])
#     best_nn_model.fit(X_combined, y_combined, batch_size=best_params['batch_size'], epochs=best_params['epochs'], verbose=0)

#     # In-sample (combined Year 1 + Year 2) predictions
#     y_combined_pred = best_nn_model.predict(X_combined)

#     # Evaluate In-Sample Performance (on combined Year 1 + Year 2)
#     rmse_combined = np.sqrt(mean_squared_error(y_combined, y_combined_pred))
#     r2_combined = r2_score(y_combined, y_combined_pred)
    
#     print(f"\nIn-Sample Performance for {option_type} Options (Year 1 + Year 2):")
#     print(f"RMSE (Training + Validation): {rmse_combined:.4f}")
#     print(f"R² (Training + Validation): {r2_combined:.4f}")

#     # After retraining, evaluate performance on the test data (Year 3)
#     y_test_pred = best_nn_model.predict(X_test)

#     # Evaluate Out-of-Sample Performance (on Test Data)
#     rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
#     r2_test = r2_score(y_test, y_test_pred)

#     # Print the final results
#     print(f"\nPerformance on Test Data (Year 3) for {option_type} Options:")
#     print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
#     print(f"Out-of-Sample R² (Test): {r2_test:.4f}")


# # Call the function for Call options data
# prepare_data_with_gridsearch_nn(data_train_c, data_validate_c, data_test_c, 'Call')

# # Call the function for Put options data
# prepare_data_with_gridsearch_nn(data_train_p, data_validate_p, data_test_p, 'Put')
